<font size=+3 color="#3B3534"><center><b>Housing in London 🏘️💷💂</b></center></font>

<img src="https://images.unsplash.com/photo-1501683567677-568cfd15361c?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=1414&q=80" width="750">
<center>Photo by Nigel Tadyanehondo on Unsplash</center>

<br>

# Introduction

Hello readers! The topic of this notebook is the housing market of London in the years between 1995 and 2020. We are going to perform exploratory data analysis with the goal of discovering new information and answering the following questions:

- How has the housing market (average house price and number of houses sold) changed over the years in different boroughs of London? How does it compare to England?
- What factors affected it the most?

<br>

**Table of Contents**

1. [Introduction](#Introduction)
2. [Libraries](#Libraries)
3. [Meet the Datasets](#Datasets)
4. [London Housing over the Years](#Housing-over-the-Years)
5. [Factors Affecting Housing](#Factors-Affecting-Housing)
6. [Conclusions](#Conclusions)
7. [Future Developement](#Future-Developement)

# Libraries

We start by importing the necessary libraries and setting some parameters for the whole notebook (e.g. the display format for the pandas library).

In [ ]:
import numpy as np

import pandas as pd
from pandas.plotting import scatter_matrix
import geopandas as gpd

import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
import seaborn as sns

from IPython.display import HTML

import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install gif
import gif

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('precision', 2)
font_size = 17

# Datasets

We are going to use the [Housing in London](https://www.kaggle.com/justinas/housing-in-london) dataset which is provided by [Justinas Cirtautas](https://www.kaggle.com/justinas). It contains two csv files ('**housing_in_london_monthly_variables.csv**' and '**housing_in_london_yearly_variables.csv'**) with a lot of relevant information such as the monthly average house prices, yearly mean and median salary for residents of each area, etc.

The data is split by areas of London called boroughs, but some of the instances correspond to other UK regions (like North East, West Midlands, etc.).

Additionally, we are going to use a third dataset for plotting maps ([London Borough and Ward Boundaries up to 2014](https://www.kaggle.com/csobral/london-borough-and-ward-boundaries-up-to-2014/notebooks), provided by [Cayo Costa Sobral](https://www.kaggle.com/csobral)).

# Housing over the Years

We are going to focus on the **average price** and the **number of houses sold** in each area. 

For this task, we need two datasets:
- '**housing_in_london_monthly_variables**' which contains monthly information about the London boroughs and the other UK regions, and
- '**London_Borough_Excluding_MHW.shp**' for plotting maps.

## Data Preprocessing

Let's import the first dataset and take a quick look at the data:

In [ ]:
df_m = pd.read_csv('../input/housing-in-london/housing_in_london_monthly_variables.csv', parse_dates = ['date'])

print ('This dataset contains {} rows and {} columns.'.format(df_m.shape[0], df_m.shape[1]))
df_m.head()

There are 7 attributes in total. An instance represents a London borough if 'borough_flag' equals 1. The meaning of the rest of the attributes can be easily inferred from their name.

The method `info()` can give us valuable information such as the type and the number of missing values in each attribute:

In [ ]:
df_m.info()

In [ ]:
null_df_m = df_m.isnull().sum().sort_values(ascending = False)
percent = (df_m.isnull().sum()/df_m.isnull().count()).sort_values(ascending = False)*100

null_df_m = pd.concat([null_df_m, percent], axis = 1, keys = ['Counts', '% Missing'])
print ('Missing: ')
null_df_m.head()

Two attributes, 'houses_sold' and 'no_of_crimes', have missing values. We will drop the whole 'no_of_crimes' attribute since almost 50% of its instances have NaN values. 

It's easier to replace missing values in the 'houses_sold' attribute as only a small portion is missing. We will use the mean value for the same area for all years in each case. Of course, someone could argue that this number would change over the years, but we could assume that the final results won't be affected due to small number of values being changed.

In [ ]:
df_m.drop('no_of_crimes', axis = 1, inplace = True)   # drop the 'no_of_crimes column

df_m['houses_sold'].fillna(df_m.groupby('area')['houses_sold'].transform('mean'), inplace = True) # fill NaN values with the mean of that particular area

The 'date' attribute is of datatime type, hence we can extract the corresponding year of each instance (month and day are not important):

In [ ]:
df_m['year'] = df_m['date'].dt.year
df_m.iloc[[0, -1]]

The dataset contains samples from January 1995 to January 2020. We will discard all samples from 2020 since the year is not complete:

In [ ]:
df_m = df_m[df_m['year'] < 2020]
df_m['year'].max()

## Data Exploration

According to [Wikipedia](https://en.wikipedia.org/wiki/List_of_London_boroughs), there are 33 boroughs in London (32 + the City of London). Is that the case in our dataset?

In [ ]:
lnd_boroughs = df_m[df_m['borough_flag'] == 1]['area'].unique()
len(lnd_boroughs)

It is! How many and which regions are outside of London?

In [ ]:
df_m[df_m['borough_flag'] == 0]['area'].nunique()

In [ ]:
df_m[df_m['borough_flag'] == 0]['area'].unique()

We can manually isolate the ones that correspond to a region in England: 

In [ ]:
eng_regions = ['south west', 'south east', 'east of england', 'west midlands', 'east midlands', 'yorks and the humber', 'north west', 'north east']

### London VS England

We can now split the dataset into two: one for boroughs in London and one for the other regions of England:

In [ ]:
lnd = df_m[df_m['area'].isin(lnd_boroughs)]
eng = df_m[df_m['area'].isin(eng_regions)]

#### Average Price

The `groupby()` method allows us to calculate the mean 'average price' for each 'date':

In [ ]:
lnd_pr = lnd.groupby('date')['average_price'].mean()
eng_pr = eng.groupby('date')['average_price'].mean()

In [ ]:
plt.figure(figsize = (9, 5))

lnd_pr.plot(y = 'average_price', color = 'royalblue', lw = 2, label = 'London')
eng_pr.plot(y = 'average_price', color = 'firebrick', lw = 2, label = 'England')

plt.axvspan('2007-12-21', '2009-06-21', alpha = 0.5, color = '#E57715')
plt.text(x = '2008-04-01', y = 390000, s = 'Recession', rotation = 90, fontsize = font_size-2)
plt.axvline(x = '2016-06-23', lw = 2, color = '#E57715', linestyle = '--')
plt.text(x = '2015-08-01', y = 210000, s = 'Brexit Referendum', rotation = 90, fontsize = font_size-2)

plt.title('Time evolution of the average house price', size = font_size)
plt.ylabel('Average Price', size = font_size)
plt.xticks(size = font_size - 3)
plt.xlabel('Date', size = font_size)
plt.yticks(size = font_size - 3)
plt.legend(fontsize = font_size - 3);

- In overall, the averace price follows an **upward trend** during the studied time frame, with London always having a higher average price. 
- This upward trend was disrupted by the the **[Great Recession](https://en.wikipedia.org/wiki/Great_Recession)** (2007-209), hence the significant **decline starting from 2007**. London apparently **recovered quickly** and the average price increased rapidly until 2016. **England** followed a similar behaviour but with a **moderate rise** in the same period.
- Arguably, the **[Brexit referendum](https://en.wikipedia.org/wiki/2016_United_Kingdom_European_Union_membership_referendum)** had an impact in London housing since the average price hit a **plateau after 2016**. It seems the the rest of England wasn't affected as much from the referendum.

<br>

<font color="#842A16" size=+0><b>Gifs</b></font> 

I really like adding another dimension in my graphs, by making **gifs**. I recently came across the [**gif**](https://github.com/maxhumber/gif) library by Max Humber, which allows us to make gifs simply by building a bunch of "frames" with a standard for loop.

The steps are the following:

- Create a 'plot' function which plots the average price for both London (df_lnd) and England (df_eng) until the specified date (third argument). It doesn't have to return something.
- Decorate the plot function with @gif.frame, 
- Use a for loop to create a frame for each month, and append the result to a list (called 'frames' in our case). Finally,
- Use the `save()` method from the gif library to save it.

In [ ]:
@gif.frame
def plot(df_lnd, df_eng, date):
    
    ### select a sub-dataframe from the start until date ###
    d_ln = df_lnd.loc[df_lnd.index[0]:date]
    d_eng = df_eng.loc[df_eng.index[0]:date]
    
    fig = plt.figure(figsize = (9, 5))
    plt.xlim(pd.Timestamp('1994-12-01'), pd.Timestamp('2020-01-01'))
    plt.ylim(47000, 550000)
    
    ### for the vertical orange rectangle and the vertical dashed line ###
    if (date > pd.Timestamp('2007-12-22') and date < pd.Timestamp('2009-06-21')):
        plt.axvspan(pd.Timestamp('2007-12-21'), date, alpha = 0.5, color = '#E57715') 
    elif (date > pd.Timestamp('2009-06-21')):
        plt.axvspan(pd.Timestamp('2007-12-21'), pd.Timestamp('2009-06-21'), alpha = 0.5, color = '#E57715')
        plt.text(x = pd.Timestamp('2008-04-29'), y = 390000, s = 'Recession', rotation = 90, fontsize = font_size-2)
    if (date > pd.Timestamp('2016-06-23')):
        plt.axvline(x = pd.Timestamp('2016-06-23'), lw = 2, color = '#E57715', linestyle = '--')
        plt.text(x = pd.Timestamp('2015-08-01'), y = 210000, s = 'Brexit Referendum', rotation = 90, fontsize = font_size-2)
    ############################################################################################################
    
    plt.plot(d_ln, color = 'royalblue', lw = 2, label = 'London')
    plt.plot(d_eng, color = 'firebrick', lw = 2, label = 'England')
    
    plt.title('Time evolution of the average house price', size = font_size)
    plt.ylabel('Average Price', size = font_size)
    plt.xticks(size = font_size - 3)
    plt.xlabel('Date', size = font_size)
    plt.yticks(size = font_size - 3)
    plt.legend(loc = 2, fontsize = font_size - 3);

In [ ]:
frames = []
for months in pd.date_range(start = lnd_pr.index[0], end = lnd_pr.index[-1], freq = '3MS'): # 3MS --> every three months
    frame = plot(lnd_pr, eng_pr, months)
    frames.append(frame)
    
gif.save(frames, 'Price-Lnd_vs_Eng.gif', duration = 1, unit = 's', between = 'startend')

We can insert our gif in the notebook:

In [ ]:
HTML('<img src="./Price-Lnd_vs_Eng.gif" />')

<br>

The average price in each borough fluctuates through time. However, we can calculate its mean which can give us a rough indication of how expensive each area is.

In [ ]:
lnd_b_prices = lnd.groupby('area')['average_price'].mean()
lnd_top10_pr = lnd_b_prices.sort_values(ascending = False).to_frame()

print ('\nThe 10 most expensive boroughs in London are:')
lnd_top10_pr.head(10)

In [ ]:
lnd_top10_pr.head(10).sort_values(by = 'average_price', ascending = True).plot(kind = 'barh', figsize = (9, 5), 
                                                                               color = 'steelblue', edgecolor = 'firebrick',
                                                                               legend = False)

plt.title('Average price in the most expensive London boroughs (1995-2019)', size = font_size, y = 1.05)
plt.ylabel('London Borough', size = font_size)
plt.yticks(size = font_size - 3)
plt.xlabel('Average Price', size = font_size)
plt.xticks([0, 200_000, 400_000, 600_000], size = font_size - 3);

We focus on the 5 most expensive boroughs and see the evolution of the average price through time:

In [ ]:
top5_indeces = lnd_top10_pr.head().index
colors = ['#e74c3c', '#3498db', '#95a5a6', '#34495e', '#2ecc71']

plt.figure(figsize = (9, 5))

for index, i in enumerate(top5_indeces):
    df_ = lnd[lnd['area'] == i]
    df_ = df_.groupby('date')['average_price'].mean()
    
    df_.plot(y = 'average_price', label = i, color = colors[index])
       
plt.title('Average price in the most expensive boroughs', y = 1.04, size = font_size)
plt.xlabel('Date', size = font_size)
plt.xticks(size = font_size - 3)
plt.ylabel('Average Price', size = font_size)
plt.yticks([0.2*1E+6, 0.6*1E+6, 1.0*1E+6, 1.4*1E+6], size = font_size - 3)
plt.legend(fontsize = font_size - 5);

- **Kensington and Chelsea** is the **most expensive borough** in London between 1995 and 2019. We can read in [wikipedia](https://en.wikipedia.org/wiki/Royal_Borough_of_Kensington_and_Chelsea) that it holds royal status and it includes many affluent areas, hence the increased price.
- **Westministe**r, which is adjacent to Kensighton, comes **second**. Again this is not a surpise since it is the location of the national government and includes many sites commonly associated with London (e.g. the Buckingham Palace, the Houses of Parliament, 10 Downing Street, and Trafalgar Square) ([Source](https://en.wikipedia.org/wiki/City_of_Westminster)).

We can perform a similar analysis for England:

In [ ]:
eng_prices = eng.groupby('area')['average_price'].mean()
eng_top3_pr = eng_prices.sort_values(ascending = False).to_frame()

print('The top 3 most expensive regions in England are:')
eng_top3_pr.head(3)

In [ ]:
top3_indeces = eng_top3_pr.head(3).index
colors = ['darkorange', '#8EB8E5', 'forestgreen', ]

plt.figure(figsize = (9, 5))

for index, i in enumerate(top3_indeces):
    df_ = eng[eng['area'] == i]
    df_ = df_.groupby('date')['average_price'].mean()
    df_.plot(y = 'average_price', label = i, color = colors[index])

plt.title('Average price in the most expensive English regions by date', size = font_size, y = 1.04)
plt.xlabel('Date', size = font_size)
plt.xticks(size = font_size - 3)
plt.ylabel('Average Price', size = font_size)
plt.yticks([100_000, 200_000, 300_000], size = font_size - 3)
plt.legend(fontsize = font_size - 3);

The three **most expensive English regions** are all located in the **South of England** (two of them border London). It would be interesting to plot the same graph but with the cheapest borough of London (which we can find is Barking and Dagenham):

In [ ]:
plt.figure(figsize = (9, 5))

for index, i in enumerate(top3_indeces):
    df_ = eng[eng['area'] == i]
    df_ = df_.groupby('date')['average_price'].mean()
    df_.plot(y = 'average_price', label = i, color = colors[index])

lnd_bng_pr = lnd[lnd['area'] == 'barking and dagenham'].groupby('date')['average_price'].mean()
lnd_bng_pr.plot(y = 'average_price', lw = 2, linestyle = '--', color = '#A30015', label = 'barking and dagenham')

plt.title('3 expensive English regions VS cheapest London borough', size = font_size, y = 1.06)
plt.xlabel('Date', size = font_size)
plt.xticks(size = font_size - 3)
plt.ylabel('Average Price', size = font_size)
plt.yticks([0.1*1E+6, 0.2*1E+6, 0.3*1E+6], size = font_size - 3)
plt.legend(labels = ['South East (Eng)', 'East of England (Eng)', 'South West (Eng)', 'Barking and Dagenham (L)'], 
           fontsize = font_size - 3);

It is evident that even the cheapest borough in London is **comparable** to the most expensive regions of England!

### Houses Sold

Let's explore the 'houses_sold' attribute for London.

In [ ]:
lnd_houses = lnd.groupby('date')['houses_sold'].sum()
lnd_houses.plot(figsize = (9, 5), lw = 2, y = 'houses_sold', color = '#00072D')

plt.axvspan('2007-12-21', '2009-06-21', alpha = 0.5, color = '#F08700')
plt.text(x = '2008-04-01', y = 10700, s = 'Recession', rotation = 90, fontsize = font_size-2)
plt.axvspan('2016-01-1', '2016-05-01', alpha = 0.7, color = '#FFCAAF')

# plt.axvline(x = '2016-06-23', lw = 2, color = '#E57715', linestyle = '--')
plt.text(x = '2016-06-01', y = 10000, s = 'New tax legislation', rotation = 90, fontsize = font_size-2)

plt.title('Houses sold in London by date', size = font_size)
plt.xlabel('Date', size = font_size)
plt.xticks(size = font_size - 3)
plt.ylabel('Houses sold', size = font_size)
plt.yticks([4000, 8000, 12000, 16000], size = font_size - 3);

- The number of houses sold in London **dropped sharply** during the **financial recession** and it seems that it **hasn't completely recovered** since,  
- Interestingly, there is **spike in March 2016** which at first sight looks like a artefact. That is not the case, because people were actually trying to avoid a **new legislation** that came into effect in April 2016 and imposed an increase in the tax bill on buying a second home (Credits to [Marcel Fellipe](https://www.kaggle.com/marcelfellipe/housing-in-london-exploratory-data-analysis) for finding this). 
- Again, the **referendum** resulted in a **downward trend** starting from July 2016.

<br>

The boroughs with the highest number of sold houses between 1995 and 2019 are:

In [ ]:
lnd_b_houses = lnd.groupby('area')['houses_sold'].sum()
lnd_top5_h = lnd_b_houses.sort_values(ascending = False).to_frame()
lnd_top5_h.head(5)

To better understand these numbers we need more information such as the size of the borough, its population, crime rates etc. Some of these information are available in the other dataset ('housing_in_london_yearly_variables').

## Map of London

We are going to use the **second dataset** which contains geospatial information about London (specifically, the 'geometry' attribute). This [notebook](https://www.kaggle.com/justinas/house-prices-in-london) helped me a lot for the analysis.

We start by merging the two datasets:

In [ ]:
london_map = gpd.read_file('../input/london-borough-and-ward-boundaries-up-to-2014/London_Wards/Boroughs/London_Borough_Excluding_MHW.shp')
london_map.columns = london_map.columns.str.lower()
london_map.head()

In [ ]:
london_map['name'] = london_map['name'].str.lower()
london_map.rename(columns = {'name': 'area'}, inplace = True)
london_map.rename(columns = {'gss_code': 'code'}, inplace = True)

london_map = london_map[['area', 'code', 'hectares', 'geometry']]
london_map.head()

In [ ]:
lnd_m = lnd.groupby('area').agg({'average_price': ['mean'], 'houses_sold': 'sum'})

lnd_m.columns = ['average_price', 'houses_sold']
lnd_m.reset_index(inplace = True)
lnd_m.head()

Remember that London has 33 boroughs, same as the lnd_tot dataset. Let's check that the london_map dataset has the same names:

In [ ]:
np.intersect1d(lnd_m['area'], london_map['area']).size

It does, so we are good to go.

In [ ]:
lnd_m_map = pd.merge(london_map, lnd_m, how = 'inner', on = ['area'])
lnd_m_map.head()

In [ ]:
type(lnd_m_map)

The merged dataset has a new type, **geodataframe**, which is what we need for plotting the map.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (12, 12))

lnd_m_map.plot(ax = ax[0], column = 'average_price', cmap = 'Reds', edgecolor = 'maroon', legend = True, legend_kwds = {'label': 'Average Price', 'orientation' : 'horizontal'})

lnd_m_map.plot(ax = ax[1], column = 'houses_sold', cmap = 'Blues', edgecolor = 'maroon', legend = True, legend_kwds = {'label': 'Houses Sold', 'orientation' : 'horizontal'})

ax[0].axis('off')
ax[0].set_title('Average House Price (All years)', size = font_size)
ax[1].axis('off')
ax[1].set_title('Houses Sold (All years)', size = font_size);

- We can now see Kensighton and Westminister as the two dark boroughs in the first map. We can roughly say that boroughs **closer** to these two areas are **more expensive**.
- As we hinted earlier, the number of houses sold is probably affected by many factors with one of them being the **total area** of each borough. We can definetely see that larger boroughs have more houses sold.

---

# Factors Affecting Housing

## Preparing the Final Dataset

For this question we need to import the third and final dataset ('**housing_in_london_yearly_variables**') which we are going to merge with the monthly dataset.

In [ ]:
df_y = pd.read_csv('../input/housing-in-london/housing_in_london_yearly_variables.csv', parse_dates = ['date'])
df_y = df_y[df_y['area'].isin(lnd_boroughs)] # select only London boroughs

print ('This dataset contains {} rows and {} columns.'.format(df_y.shape[0], df_y.shape[1]))
df_y.head()

This dataset has more information for each borough and is ideal for answering the second question. Let's see how many missing values each attrbitutes has:

In [ ]:
null_df_y = df_y.isnull().sum().sort_values(ascending = False)
percent = (df_y.isnull().sum()/df_y.isnull().count()).sort_values(ascending = False)*100

null_df = pd.concat([null_df_y, percent], axis = 1, keys = ['Counts', '%'])
null_df.head(10)

In [ ]:
# import missingno as msno
# msno.matrix(df_y)

It seems that all new attributes have missing values. Pandas' `corr()` function, which we are going to use later, ignores a pairwise correlation if there is a NaN value in one of the observations, so we shouldn't worry about it for now.

'median_salary' is not currently numeric. If we try to convert it to numeric (with the `astype()` method), we'll get an error. That's because some instances contain symbols and need to be replaced first:

In [ ]:
df_y[~df_y['mean_salary'].str.isnumeric()]['mean_salary'].value_counts()

In [ ]:
df_y['mean_salary'] = df_y['mean_salary'].replace(['#'], np.NaN)
df_y['mean_salary'] = df_y['mean_salary'].astype(float)

In [ ]:
df_y['year'] = df_y['date'].dt.year

print ('yearly_variables dataset')
print ('\tFirst date: ', df_y['year'].min())
print ('\tFinal date: ', df_y['year'].max())

This dataset starts four years later compared to the monthly dataset. Therefore, we need to merge them so that there is no mismatch in the 'year' column:

In [ ]:
lnd_m_group = lnd.groupby(['area', 'year']).mean().reset_index()  # group based on area and year (take mean)
lnd_m_group = lnd_m_group[lnd_m_group['year'] >= 1999]            # select all years after 1999 (included)

print ('monthly_variables dataset')
print ('\tFirst date: ', lnd_m_group['year'].min())
print ('\tFinal date: ', lnd_m_group['year'].max())

In [ ]:
lnd_y_group = df_y.groupby(['area', 'year']).mean().reset_index() # group it based on area and year
lnd_y_group.head()

The two datasets are ready to be merged:

In [ ]:
lnd_total = pd.merge(lnd_y_group, lnd_m_group, on = ['area', 'year'], how = 'left')
lnd_total.drop(['borough_flag_x', 'borough_flag_y'], axis = 1, inplace = True)

lnd_total.head()

## Correlations

In [ ]:
corr_table = lnd_total.corr()
corr_table['average_price'].sort_values(ascending = False)

- It doesn't come as a surpise that apart from the year, the average price in London has a **significant positive correlation** with the **median and mean salary** along with the **number of jobs** in a borough. 
- There is **small positive correlation** with **life satisfaction**. More data for this attribute would be useful.
- As we discussed earlier, the most affluent boroughs happen to be small (see map) which explains the **negative correlation** with the **size** of an area. 
- The average price is also affected by the **number of houses sold**, since the former goes up when the later decreases (people sell higher when less houses are sold?).

We can summarise these correlations (along with all pair-wise correlations) with the following heatmap:

In [ ]:
plt.figure(figsize = (10, 8))

mask = np.triu(np.ones_like(corr_table, dtype = np.bool))

ax = sns.heatmap(corr_table, mask = mask, annot = True, cmap = 'YlGnBu_r')
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5);

We can see some obvious correlations such as median-mean salary, houses sold-no_of_houses, no_of_houses-population_size, etc.

Lastly, a linear relationship between attributes can be visualised with the `scatter_matrix` method:

In [ ]:
columns = ['average_price', 'median_salary', 'mean_salary', 'number_of_jobs']

scatter_matrix(lnd_total[columns], figsize = (12, 12), color = '#D52B06', alpha = 0.3, 
               hist_kwds = {'color':['bisque'], 'edgecolor': 'firebrick'});

# Conclusions

The main conclusions are the following:

- The average price in **London boroughs** is **higher** compared to the rest of England. It has been **affected** by major financial and political events (such the Recession and Brexit), but has **significantly increased** from 1995 to 2019 (by a factor of 5),

- **Affluent regions** such as Kensighton and Westiminister have the **highest average price**, 

- The number of houses sold **plummeted** after the recession, **streadily increased** until 2016 but then again **dropped** after the referendum, 

- As anyone could predict, the main factor that influences the average price is the **financial prosperity** of the corresponding borough. Higher salaries and more jobs result in higher prices.

# Future Developement

This notebook could be further improved by making some additions:

- Use maps to visualise how variables in the 'yearly_dataset' vary from borough to borough,
- Compare English regions with London (all boroughs) with respect to the other variables (maps could be useful),
- Use Folium for plotting interactive maps.
    
<br>
Please <font color="red" size=+0><b>upvote</b></font> if you liked this notebook! Thank you! 😉